In [1]:
pip install pandas sklearn sportsreference

     |████████████████████████████████| 472 kB 5.2 MB/s 


In [2]:
import os
import numpy as np
import pandas as pd
import regex as re

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
from sportsreference.ncaab.teams import Teams

In [5]:
list_of_teams = Teams()

In [6]:
heights = pd.read_csv("/content/drive/MyDrive/Capstone Project/2022WideData/height22.csv")
miscDef = pd.read_csv("/content/drive/MyDrive/Capstone Project/2022WideData/DefenseMisc22.csv")
miscOff = pd.read_csv("/content/drive/MyDrive/Capstone Project/2022WideData/OffenseMisc22.csv")

miscOff=miscOff.add_prefix("Offensive_")
miscOff= miscOff.rename(columns={"Offensive_Season":"Season", "Offensive_TeamName":"TeamName"})

miscDef = miscDef.add_prefix("Defensive_")
miscDef = miscDef.rename(columns={"Defensive_Season":"Season","Defensive_TeamName":"TeamName"})

final_data_frame = pd.concat([miscOff,miscDef,heights],axis=1)
final_data_frame = final_data_frame.loc[:,~final_data_frame.columns.duplicated()]

In [7]:
offensive_four_factors_2021_2022 = pd.read_csv("/content/drive/MyDrive/Capstone Project/2022WideData/offense22.csv")
defensive_four_factors_2021_2022 = pd.read_csv("/content/drive/MyDrive/Capstone Project/2022WideData/defense22.csv")

# Main Functions

In [8]:
# def tossup(team_one,team_two):
#   print("TOSS UP")
#   dataframe_1 = final_dict_schedules[team_one]
#   dataframe_2 = final_dict_schedules[team_two]

#   dataframe_1_select = dataframe_1.iloc[:,11:23]
#   dataframe_2_select = dataframe_2.iloc[:,11:23]


#   dataframe_1_select = dataframe_1.iloc[:,[1,4,6,7,8,9,11,22]]
#   dataframe_2_select = dataframe_2.iloc[:,[1,4,6,7,8,9,11,22]]

#   new_column_names_1  = []
#   for names in dataframe_1_select.columns:
#     if names == "Final Point Differential":
#       new_column_names_1.append(names)
#     else:
#       names = names.replace("tm","Opp")
#       new_column_names_1.append(names)
        
#   dataframe_1_select.columns = new_column_names_1
#   new_column_names_2  = []
#   for names in dataframe_2_select.columns:
#     if names == "Final Point Differential":
#       new_column_names_2.append(names)
#     else:
#       new_column_names_2.append(names)
#   dataframe_2_select.columns = new_column_names_2
#   d_1 = dataframe_1_select.join(dataframe_2_select, on="Final Point Differential", rsuffix="Other ")
#   column_means = d_1.mean()
#   d_1 = d_1.fillna(column_means)
#   d_1 = d_1.drop(['Final Point DifferentialOther ','Final Point Differential'],axis=1)
#   return d_1
#   trial_1 = perform_log_prediction(final_dict_schedules[team_two].iloc[:,[1,4,6,7,8,9,22,23]],d_1)

#   dataframe_1 = final_dict_schedules[team_two]
#   dataframe_2 = final_dict_schedules[team_one]

#   dataframe_1_select = dataframe_1.iloc[:,11:23]
#   dataframe_2_select = dataframe_2.iloc[:,11:23]


#   dataframe_1_select = dataframe_1.iloc[:,[1,4,6,7,8,9,11,22]]
#   dataframe_2_select = dataframe_2.iloc[:,[1,4,6,7,8,9,11,22]]



#   new_column_names_1  = []
#   for names in dataframe_1_select.columns:
#     if names == "Final Point Differential":
#       new_column_names_1.append(names)
#     else:
#       names = names.replace("tm","Opp")
#       new_column_names_1.append(names)
        
#   dataframe_1_select.columns = new_column_names_1
#   new_column_names_2  = []
#   for names in dataframe_2_select.columns:
#     if names == "Final Point Differential":
#       new_column_names_2.append(names)
#     else:
#       new_column_names_2.append(names)
#   dataframe_2_select.columns = new_column_names_2
#   d_2 = dataframe_1_select.join(dataframe_2_select, on="Final Point Differential", rsuffix="Other ")
#   column_means = d_2.mean()
#   d_2 = d_2.fillna(column_means)
#   d_2 = d_2.drop(['Final Point DifferentialOther ','Final Point Differential'],axis=1)
#   trial_2 = perform_log_prediction(final_dict_schedules[team_one].iloc[:,[1,4,6,7,8,9,22,23]],d_2)
#   print(trial_1[0])
#   print(trial_2[0])
#   if trial_1[0]==0 and trial_2[0]==1:
#     print(team_one, " won!")
#     return team_one
#   elif trial_2[0]==0 and trial_1[0]==1:
#     print(team_two, " won!")
#     return team_two
#   elif trial_1[0]==0 and trial_2[0]==0 or trial_1[0]==1 and trial_2[0]==1:
#     print("This is a double tossup")
#     winner = random.randint(0, 1)
#     if winner==0:
#       print(team_one, " won!")
#       return team_one
#     else:
#       print(team_two, " won!")
#       return team_two

In [9]:
def clean_opponent_name(dataframe):
  clean_opponent_list = []
  pattern = r"[(\d)]"
  for team in dataframe.index:
    clean_opponent = re.sub(pattern,"",dataframe["opponent_name"][team])
    clean_opponent = clean_opponent.replace("()","")
    if clean_opponent == "North Carolina State":
      clean_opponent = "N.C. State"
    if clean_opponent =="Florida State" or clean_opponent == "Florida State ":
      clean_opponent ="Florida St."
    
    clean_opponent = clean_opponent.strip()
    clean_opponent_list.append(clean_opponent)
  dataframe["Official Opponent"] = clean_opponent_list
  dataframe = dataframe.drop(columns=["opponent_name"],axis=1)
  return dataframe

In [10]:
def grab_stats(offensive_factors, defensive_factors, team_name):
  offense_factors = offensive_factors[offensive_factors["TeamName"]== team_name]
  offense_factors = offense_factors.drop(["RankeFGPct","RankTOPct","RankORPct","RankFTRate","Season", "TeamName"],axis =1)
  defense_factors = defensive_factors[defensive_factors["TeamName"]== team_name]
  defense_factors = defense_factors.drop(["RankeFGPct","RankTOPct","RankORPct","RankFTRate","Season","TeamName"],axis =1)
  return offense_factors.values.tolist()[0], defense_factors.values.tolist()[0]

In [11]:
def grab_misc_stats(dataframe, team_name):
  dataframe = dataframe[dataframe.columns.drop(list(dataframe.filter(regex='Rank')))]
  misc_stats = dataframe[dataframe["TeamName"] == team_name]
  return misc_stats[["Exp","Bench","Size"]].values.tolist()

In [12]:
def return_final_dataframe(dataframe,team_name):
  offense_stats, defense_stats = grab_stats(offensive_four_factors_2021_2022,defensive_four_factors_2021_2022,team_name)
  tm_misc_stats = grab_misc_stats(final_data_frame,team_name)
  new_dataframe = dataframe[["Unnamed: 4","Official Opponent","Tm","Opp"]]
  new_dataframe[["Opp Off eFG%","Opp Off TOP%","Opp Off ORB%", "Opp Off FTR%"]] = np.NaN
  new_dataframe[["Opp Def eFG%","Opp Def TOP%","Opp Def ORB%", "Opp Def FTR%"]] = np.NaN
  for index,row in new_dataframe.iterrows():
    try:
      opponent_off_fac, opponent_def_fac = grab_stats(offensive_four_factors_2021_2022, defensive_four_factors_2021_2022, row["Official Opponent"])
      opponent_misc_stats = grab_misc_stats(final_data_frame, row["Official Opponent"])
      
      final_list = opponent_off_fac + opponent_def_fac + opponent_misc_stats[0]
      new_dataframe.at[ index, "Opp Off eFG%"] = final_list[0]
      new_dataframe.at[ index, "Opp Off TOP%"] = final_list[1]
      new_dataframe.at[ index, "Opp Off ORB%"] = final_list[2]
      new_dataframe.at[ index, "Opp Off FTR%"] = final_list[3]
      new_dataframe.at[ index, "Opp Def eFG%"] = final_list[4]
      new_dataframe.at[ index, "Opp Def TOP%"] = final_list[5]
      new_dataframe.at[ index, "Opp Def ORB%"] = final_list[6]
      new_dataframe.at[ index, "Opp Def FTR%"] = final_list[7]
      new_dataframe.at[ index, "Opp Exp"] = final_list[8]
      new_dataframe.at[ index, "Opp Bench"] = final_list[9]
      new_dataframe.at[ index, "Opp Size"] = final_list[10]
    except IndexError:
      continue
  total_stats = offense_stats + defense_stats + tm_misc_stats[0]
  new_dataframe["tm Off eFG%"] = total_stats[0]
  new_dataframe["tm Off TOP%"] = total_stats[1]
  new_dataframe["tm Off ORB%"] = total_stats[2]
  new_dataframe["tm Off FTR%"] = total_stats[3]
  new_dataframe["tm Def eFG%"] = total_stats[4]
  new_dataframe["tm Def TOP%"] = total_stats[5]
  new_dataframe["tm Def ORB%"] = total_stats[6]
  new_dataframe["tm Def FTR%"] = total_stats[7]
  new_dataframe["tm Exp"] = total_stats[8]
  new_dataframe["tm Bench"] = total_stats[9]
  new_dataframe["tm Size"] = total_stats[10]
  new_dataframe["Final Point Differential"] = new_dataframe["Tm"] - new_dataframe["Opp"]
  new_dataframe = new_dataframe.drop(columns=['Unnamed: 4','Official Opponent'])
  new_dataframe['BinaryResult'] = np.where(new_dataframe['Final Point Differential'] > 0 , 1, 0)
  new_dataframe = new_dataframe.dropna()
  new_dataframe = new_dataframe.drop(['Tm'], axis=1)
  new_dataframe = new_dataframe.drop(['Opp'], axis=1)
  return new_dataframe

# New Data

In [13]:
list_team_names = []
for team in list_of_teams:
  if team.name == "Brigham Young":
    list_team_names.append("BYU")
  elif team.name == "Central Florida":
    list_team_names.append("UCF")
  elif team.name == "Florida International":
    list_team_names.append("FIU")
  elif team.name == "Long Island University":
    list_team_names.append("LIU")
  elif team.name == "Louisiana":
    list_team_names.append("Louisiana State")
  elif team.name == "Maryland-Baltimore County":
    list_team_names.append("UMBC")
  elif team.name == "Massachusetts-Lowell":
    list_team_names.append("UMass Lowell")
  elif team.name == "Kansas City":
    list_team_names.append("UMKC")
  elif team.name == "Nevada-Las Vegas":
    list_team_names.append("Nevada")
  elif team.name == "NC State":
    list_team_names.append("N.C. State")
  elif team.name == "Saint Francis (PA)":
    list_team_names.append("St. Francis PA")
  elif team.name == "Saint Mary's (CA)":
    list_team_names.append("Saint Mary's")
  elif team.name == "South Carolina Upstate":
    list_team_names.append("USC Upstate")
  elif team.name == "Southern California":
    list_team_names.append("USC")
  elif team.name == "Southern Methodist":
    list_team_names.append("SMU")
  elif team.name == "Southern Mississippi":
    list_team_names.append("Southern Miss")
  elif team.name == "St. Bonaventure":
    list_team_names.append("State Bonaventure")
  elif team.name == "St. Francis (NY)":
    list_team_names.append("State Francis (NY)")
  elif team.name == "Texas A&M-Corpus Christi":
    list_team_names.append("Texas A&M")
  elif team.name == "Texas-Rio Grande Valley":
    list_team_names.append("UT Rio Grande Valley")
  elif team.name == "Virginia Commonwealth":
    list_team_names.append("VCU")
  else:
    list_team_names.append(team.name)

In [14]:

final_dict_schedules = {}
failed_names = []
for team_names in range(len(list_team_names)):
  try:
    basic_df = pd.read_csv("/content/drive/MyDrive/Capstone Project/2022Data/{}.csv".format(list_team_names[team_names]))
    cleaned = clean_opponent_name(basic_df)
    cleaned = cleaned.rename(columns={"location":"Unnamed: 4","points_for":"Tm","points_against":"Opp"})
    #print(list_team_names[team_names])
    final_cleaned = return_final_dataframe(cleaned,f"{list_team_names[team_names]}")
    final_dict_schedules[list_team_names[team_names]] = final_cleaned
  except:
    failed_names.append(list_team_names[team_names])
    print(list_team_names[team_names])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [15]:
#Try and find the features that had the most impact on it

# Logistic Regression

In [16]:
from sklearn.metrics import f1_score
from  sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
def perform_log(df):
  new_df = df.drop(columns='Final Point Differential')
  train_set, test_set = train_test_split(new_df,test_size = 0.2, random_state = 9)
  y = train_set["BinaryResult"].copy()
  x = train_set.drop(columns="BinaryResult")
  
  log_reg = LogisticRegressionCV(cv=4,random_state=0, max_iter=1000).fit(x,y)
  
  log_basketball_predictions = log_reg.predict(x)
  scores = f1_score(y, log_basketball_predictions)
  log_mse = log_loss(y, log_basketball_predictions)

  importance = log_reg.coef_[0]
  return log_mse , scores,importance

In [ ]:
final_dict_schedules["Auburn"].columns

Index(['Opp Off eFG%', 'Opp Off TOP%', 'Opp Off ORB%', 'Opp Off FTR%',
       'Opp Def eFG%', 'Opp Def TOP%', 'Opp Def ORB%', 'Opp Def FTR%',
       'Opp Exp', 'Opp Bench', 'Opp Size', 'tm Off eFG%', 'tm Off TOP%',
       'tm Off ORB%', 'tm Off FTR%', 'tm Def eFG%', 'tm Def TOP%',
       'tm Def ORB%', 'tm Def FTR%', 'tm Exp', 'tm Bench', 'tm Size',
       'Final Point Differential', 'BinaryResult'],
      dtype='object')

In [ ]:
error,scoresm,importance =perform_log(final_dict_schedules["Auburn"])

In [ ]:
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: -0.00000
Feature: 1, Score: 0.00047
Feature: 2, Score: -0.00055
Feature: 3, Score: -0.00087
Feature: 4, Score: 0.00065
Feature: 5, Score: -0.00110
Feature: 6, Score: -0.00014
Feature: 7, Score: -0.00019
Feature: 8, Score: -0.00007
Feature: 9, Score: -0.00018
Feature: 10, Score: 0.00018
Feature: 11, Score: 0.00001
Feature: 12, Score: 0.00000
Feature: 13, Score: 0.00001
Feature: 14, Score: 0.00001
Feature: 15, Score: 0.00001
Feature: 16, Score: 0.00000
Feature: 17, Score: 0.00001
Feature: 18, Score: 0.00001
Feature: 19, Score: 0.00000
Feature: 20, Score: 0.00001
Feature: 21, Score: 0.00002


In [17]:

log_reg_errors = []
f1_scores_list = []
importance_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for name in final_dict_schedules.keys():
  #print(name)
  if name == "Delaware State" or name == "Lamar":
    continue
  log_error, values,importance = perform_log(final_dict_schedules[name])
  importance_list = [sum(i) for i in zip(importance, importance_list)]
  
  log_reg_errors.append(log_error)
  f1_scores_list.append(values)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selecti

In [ ]:
for i,v in enumerate(importance_list):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: -25.13313
Feature: 1, Score: 41.07369
Feature: 2, Score: -19.76595
Feature: 3, Score: -13.26620
Feature: 4, Score: 37.32968
Feature: 5, Score: -4.08651
Feature: 6, Score: 14.56971
Feature: 7, Score: 7.46394
Feature: 8, Score: 7.62890
Feature: 9, Score: 5.85040
Feature: 10, Score: -28.48853
Feature: 11, Score: 2.83601
Feature: 12, Score: 1.00098
Feature: 13, Score: 1.62019
Feature: 14, Score: 1.91999
Feature: 15, Score: 2.84510
Feature: 16, Score: 1.30950
Feature: 17, Score: 1.71680
Feature: 18, Score: 2.24447
Feature: 19, Score: 0.10174
Feature: 20, Score: 1.52894
Feature: 21, Score: 4.05288


In [ ]:
import copy
tossup_dict = copy.deepcopy(final_dict_schedules)

In [ ]:
tossup_dict["Virginia"].iloc[:,[0,1,2,4,6,10,22,23]]

,Opp Off eFG%,Opp Off TOP%,Opp Off ORB%,Opp Def eFG%,Opp Def ORB%,Opp Size,Final Point Differential,BinaryResult
0,47.482639,19.856783,33.820841,47.256473,24.783862,76.41,-8.0,0
1,46.512408,20.582513,28.944619,51.952278,25.899281,76.33,21.0,1
2,53.552180,17.123859,37.801832,43.523171,27.332783,77.41,-20.0,0
3,46.029173,19.466570,20.080321,49.256321,31.990330,76.50,16.0,1
4,50.173913,20.679468,25.830959,54.552314,30.081301,77.58,10.0,1
5,50.268496,17.328556,30.501931,46.711260,27.985740,77.71,18.0,1
6,52.085747,19.208474,21.485149,49.261084,26.411658,77.64,18.0,1
7,53.612589,12.950010,32.248062,49.787033,30.157480,77.49,-1.0,0
8,46.687117,21.169815,30.451489,52.006881,26.133076,78.29,1.0,1
9,51.080737,17.715979,25.889968,53.320802,31.521739,76.13,-3.0,0


In [ ]:
for key, value in tossup_dict.items():
  tossup_dict[key] = value.iloc[:,[0,1,2,4,6,10,22,23]]

In [ ]:
# tossup_log_reg_errors = []
# tossup_f1_scores_list = []
# tossup_importance_list = [0,0,0,0,0,0]
# for name in tossup_dict.keys():
#   #print(name)
#   if name == "Delaware State" or name == "Lamar":
#     continue
#   log_error, values,importance = perform_log(tossup_dict[name])
#   tossup_importance_list = [sum(i) for i in zip(importance, tossup_importance_list)]
  
#   tossup_log_reg_errors.append(log_error)
#   tossup_f1_scores_list.append(values)

In [ ]:
# for i,v in enumerate(tossup_importance_list):
# 	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
#np.mean(tossup_f1_scores_list)

In [ ]:
#np.mean(f1_scores_list)

# Linear Regression

In [18]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [19]:
def perform_linear_regression(df):
  df = df.drop(columns="BinaryResult")
  train_set, test_set = train_test_split(df,test_size = 0.2, random_state = 9)
  y = train_set["Final Point Differential"].copy()
  x = train_set.drop(columns="Final Point Differential")
  y_test = test_set["Final Point Differential"].copy()
  x_test = test_set.drop(columns="Final Point Differential")
  lin_reg = LinearRegression()
  lin_reg.fit(x,y)
  basketball_predictions = lin_reg.predict(x_test)
  lin_mse = mean_squared_error(y_test, basketball_predictions)
  #sklearn.metrics.f1_score(y_test, basketball_predictions)
  lin_rmse = np.sqrt(lin_mse)
  return lin_rmse

In [20]:
final_dict_schedules["Auburn"]

,Opp Off eFG%,Opp Off TOP%,Opp Off ORB%,Opp Off FTR%,Opp Def eFG%,Opp Def TOP%,Opp Def ORB%,Opp Def FTR%,Opp Exp,Opp Bench,...,tm Off FTR%,tm Def eFG%,tm Def TOP%,tm Def ORB%,tm Def FTR%,tm Exp,tm Bench,tm Size,Final Point Differential,BinaryResult
0,52.428321,20.366400,30.708661,28.964307,46.528157,17.345547,29.525483,23.674139,1.58,26.61,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,23.0,1
1,50.214988,18.946455,24.688797,26.044226,52.859779,19.406072,33.437827,34.686347,1.87,26.11,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,28.0,1
2,41.284916,19.063371,29.889924,25.418994,48.322349,18.916353,27.037392,33.553026,0.87,37.67,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,6.0,1
3,49.593909,17.356651,37.881356,30.609137,45.713511,17.950431,27.464195,31.904504,2.15,28.00,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,-6.0,0
5,52.463054,15.383278,29.475437,28.275862,50.624100,16.730402,32.105685,23.523764,2.39,19.84,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,21.0,1
7,50.089552,18.419108,25.790514,32.716418,48.854289,18.223680,26.542056,31.081081,1.97,33.94,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,22.0,1
8,50.346667,17.130074,20.683611,33.333333,51.533894,18.280601,31.390508,29.440871,1.42,32.35,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,31.0,1
9,43.241552,17.048920,25.531915,23.842303,50.204638,18.912789,29.865772,39.972715,1.12,38.87,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,26.0,1
10,50.794057,18.814102,33.801580,32.838115,47.745149,17.768874,26.887967,32.878867,1.03,25.45,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,4.0,1
11,52.939481,17.123944,35.531496,31.066282,48.004988,20.957663,25.400000,24.625935,1.59,26.06,...,31.628136,44.120763,20.717131,29.364441,34.798729,1.34,35.1,77.55,13.0,1


In [21]:
linear_errors = []
for name in final_dict_schedules.keys():
  error = perform_linear_regression(final_dict_schedules[name])
  linear_errors.append(error)

In [22]:
np.mean(linear_errors)

372720117913.6178

# Prediction

In [23]:
pip install bracketology

  Created wheel for bracketology: filename=bracketology-0.0.99-py3-none-any.whl size=30416 sha256=bb8678d06dff15b3483b89a812d4fcdea90e24cd47d0d9e6bde0c1c93ffad1a5
  Stored in directory: /root/.cache/pip/wheels/ac/9e/6e/acfdd77b5a9406ac5ef7cfe513053c6ee00086b1cf1980d19a
Successfully built bracketology


In [24]:
from bracketology import Bracket, Game, Team

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
from sklearn.linear_model import LogisticRegressionCV
def perform_log_prediction(prediction_data,test_data):
  prediction_data = prediction_data.drop(columns=["Final Point Differential"])
  train_set, test_set = train_test_split(prediction_data,test_size = 0.2, random_state = 9)
  y = train_set["BinaryResult"].copy()
  x = train_set.drop(columns="BinaryResult")
  #test_data = test_data.drop(columns=["Final Point Differential", "Final Point DifferentialOther "])
  log_reg = LogisticRegressionCV(cv=2,random_state=0, max_iter=1000).fit(x,y)

  
  log_basketball_predictions = log_reg.predict(test_data)
  
  return log_basketball_predictions

In [27]:
from sklearn.linear_model import LogisticRegressionCV
def perform_special_log_prediction(prediction_data,test_data):
  prediction_data = prediction_data.iloc[:,[0,1,2,4,6,10,22,23]]
  prediction_data = prediction_data.drop(columns=["Final Point Differential"])
  train_set, test_set = train_test_split(prediction_data, test_size = 0.2, random_state = 9)
  y = train_set["BinaryResult"].copy()
  x = train_set.drop(columns="BinaryResult")
  test_data = test_data.iloc[:,[0,1,2,4,6,10]]
  #test_data = test_data.drop(columns=["Final Point Differential", "Final Point DifferentialOther "])
  log_reg = LogisticRegressionCV(cv=2,random_state=0, max_iter=1000).fit(x,y)

  
  log_basketball_predictions = log_reg.predict(test_data)
  
  return log_basketball_predictions

# The first argument is the challenger to the higher seed

In [28]:
import random
def chose_team(team_one, team_two):
  High = Team(name=team_one,seed=1)
  # Low seed is labeled Tm
  Low = Team(name=team_two,seed=4)

  # Create a game between the teams
  game = Game(High, Low, round_number=1)
  team1 = game.top_team
  team2 = game.bottom_team

  dataframe_1 = final_dict_schedules[team1.name]
  dataframe_2 = final_dict_schedules[team2.name]
    
  dataframe_1_select = dataframe_1.iloc[:,11:23]
  dataframe_2_select = dataframe_2.iloc[:,11:23]

  new_column_names_1  = []
  for names in dataframe_1_select.columns:
    if names == "Final Point Differential":
      new_column_names_1.append(names)
    else:
      names = names.replace("tm","Opp")
      new_column_names_1.append(names)
        
  dataframe_1_select.columns = new_column_names_1
  new_column_names_2  = []
  for names in dataframe_2_select.columns:
    if names == "Final Point Differential":
      new_column_names_2.append(names)
    else:
      new_column_names_2.append(names)
  dataframe_2_select.columns = new_column_names_2
  d_1 = dataframe_1_select.join(dataframe_2_select, on="Final Point Differential", rsuffix="Other ")
  column_means = d_1.mean()
  d_1 = d_1.fillna(column_means)
  d_1 = d_1.drop(['Final Point DifferentialOther ','Final Point Differential'],axis=1)
  trial_1 = perform_log_prediction(final_dict_schedules[team_two],d_1)


  dataframe_1 = final_dict_schedules[team2.name]
  dataframe_2 = final_dict_schedules[team1.name]
    
  dataframe_1_select = dataframe_1.iloc[:,11:23]
  dataframe_2_select = dataframe_2.iloc[:,11:23]

  new_column_names_1  = []
  for names in dataframe_1_select.columns:
    if names == "Final Point Differential":
      new_column_names_1.append(names)
    else:
      names = names.replace("tm","Opp")
      new_column_names_1.append(names)
        
  dataframe_1_select.columns = new_column_names_1
  new_column_names_2  = []
  for names in dataframe_2_select.columns:
    if names == "Final Point Differential":
      new_column_names_2.append(names)
    else:
      new_column_names_2.append(names)
  dataframe_2_select.columns = new_column_names_2
  d_2 = dataframe_1_select.join(dataframe_2_select, on="Final Point Differential", rsuffix="Other ")
  column_means = d_2.mean()
  d_2 = d_2.fillna(column_means)
  d_2 = d_2.drop(['Final Point DifferentialOther ','Final Point Differential'],axis=1)
  trial_2 = perform_log_prediction(final_dict_schedules[team_one],d_2)

  if trial_1[0]==0 and trial_2[0]==1:
    print(team_one, " won!")
    return team_one
  elif trial_2[0]==0 and trial_1[0]==1:
    print(team_two, " won!")
    return team_two
  elif trial_1[0]==0 and trial_2[0]==0 or trial_1[0]==1 and trial_2[0]==1:
    print("This is a tossup")
    tossup_result = toss_up(team_one,team_two)



In [29]:
def toss_up(team_one, team_two):

  dataframe_1 = final_dict_schedules[team_one]
  dataframe_2 = final_dict_schedules[team_two]
    
  dataframe_1_select = dataframe_1
  dataframe_2_select = dataframe_2
  new_column_names_1  = []
  for names in dataframe_1_select.columns:
    if names == "Final Point Differential":
      new_column_names_1.append(names)
    else:
      names = names.replace("tm","Opp")
      new_column_names_1.append(names)
        
  dataframe_1_select.columns = new_column_names_1
  new_column_names_2  = []
  for names in dataframe_2_select.columns:
    if names == "Final Point Differential":
      new_column_names_2.append(names)
    else:
      new_column_names_2.append(names)
  dataframe_2_select.columns = new_column_names_2
  d_1 = dataframe_1_select.join(dataframe_2_select, on="Final Point Differential", rsuffix="Other ")
  column_means = d_1.mean()
  d_1 = d_1.fillna(column_means)
  d_1 = d_1.drop(['Final Point DifferentialOther ','Final Point Differential'],axis=1)
  trial_1 = perform_special_log_prediction(final_dict_schedules[team_two],d_1)


  dataframe_1 = final_dict_schedules[team_two]
  dataframe_2 = final_dict_schedules[team_one]
    
  dataframe_1_select = dataframe_1
  dataframe_2_select = dataframe_2

  new_column_names_1  = []
  for names in dataframe_1_select.columns:
    if names == "Final Point Differential":
      new_column_names_1.append(names)
    else:
      names = names.replace("tm","Opp")
      new_column_names_1.append(names)
        
  dataframe_1_select.columns = new_column_names_1
  new_column_names_2  = []
  for names in dataframe_2_select.columns:
    if names == "Final Point Differential":
      new_column_names_2.append(names)
    else:
      new_column_names_2.append(names)
  dataframe_2_select.columns = new_column_names_2
  d_2 = dataframe_1_select.join(dataframe_2_select, on="Final Point Differential", rsuffix="Other ")
  column_means = d_2.mean()
  d_2 = d_2.fillna(column_means)
  d_2 = d_2.drop(['Final Point DifferentialOther ','Final Point Differential'],axis=1)
  trial_2 = perform_special_log_prediction(final_dict_schedules[team_one],d_2)

  if trial_1[0]==0 and trial_2[0]==1:
    print(team_one, " won!")
    return team_one
  elif trial_2[0]==0 and trial_1[0]==1:
    print(team_two, " won!")
    return team_two
  elif trial_1[0]==0 and trial_2[0]==0 or trial_1[0]==1 and trial_2[0]==1:
    print("This is a double tossup")
    winner = random.randint(0, 1)
    if winner==0:
      print(team_one, " won!")
      return team_one
    else:
      print(team_two, " won!")
      return team_two

'Opp Off eFG%', 'Opp Off TOP%', 'Opp Off ORB%', 'Opp Off FTR%',
       'Opp Def eFG%', 'Opp Def TOP%', 'Opp Def ORB%', 'Opp Def FTR%',
       'Opp Exp', 'Opp Bench', 'Opp Size', 'tm Off eFG%', 'tm Off TOP%',
       'tm Off ORB%', 'tm Off FTR%', 'tm Def eFG%', 'tm Def TOP%',
       'tm Def ORB%', 'tm Def FTR%', 'tm Exp', 'tm Bench', 'tm Size',
       'Final Point Differential', 'BinaryResult'],

# First Round

### South

In [30]:
chose_team("Gonzaga","Georgia State")

Gonzaga  won!


'Gonzaga'

In [31]:
chose_team("Boise State","Memphis")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Memphis  won!


In [34]:
chose_team("Connecticut","New Mexico State")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

New Mexico State  won!


In [36]:
chose_team("Arkansas","Vermont")

Arkansas  won!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


'Arkansas'

In [38]:
chose_team("Alabama","Notre Dame")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Alabama  won!


In [39]:
chose_team("Texas Tech","Montana State")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Montana State  won!


In [40]:
chose_team("Michigan State","Davidson")

Michigan State  won!


'Michigan State'

In [42]:
chose_team("Duke","Cal State Fullerton")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Cal State Fullerton  won!


### South

In [43]:
chose_team("Baylor","Norfolk State")

Norfolk State  won!


'Norfolk State'

In [44]:
chose_team("North Carolina","Marquette")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Marquette  won!


In [45]:
chose_team("Saint Mary's","Indiana")

Saint Mary's  won!


"Saint Mary's"

In [46]:
chose_team("UCLA","Akron")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
UCLA  won!


In [47]:
chose_team("Texas","Virginia Tech")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Texas  won!


In [48]:
chose_team("Purdue","Yale")

Purdue  won!


'Purdue'

In [49]:
chose_team("Murray State","San Francisco")

Murray State  won!


'Murray State'

In [50]:
chose_team("Kentucky","Saint Peter's")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Kentucky  won!


### East

In [51]:
chose_team("Arizona","Wright State")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Arizona  won!


In [52]:
chose_team("Seton Hall","TCU")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Seton Hall  won!


In [53]:
chose_team("Houston","UAB")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Houston  won!


In [54]:
chose_team("Illinois","Chattanooga")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

Illinois  won!


In [55]:
chose_team("Colorado State","Michigan")

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Colorado State  won!


In [57]:
chose_team("Tennessee","Longwood")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Tennessee  won!


In [60]:
chose_team("Ohio State","Loyola (IL)")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Ohio State  won!


In [61]:
chose_team("Villanova","Delaware")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

Delaware  won!


### North

In [62]:
chose_team("Kansas","Texas Southern")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

Kansas  won!


In [63]:
chose_team("San Diego State","Creighton")

San Diego State  won!


'San Diego State'

In [64]:
chose_team("Iowa","Richmond")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

Iowa  won!


In [65]:
chose_team("South Dakota State","Providence")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Providence  won!


In [67]:
chose_team("Louisiana State","Iowa State")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Iowa State  won!


In [68]:
chose_team("Wisconsin","Colgate")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Wisconsin  won!


In [69]:
chose_team("USC","Miami (FL)")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
USC  won!


In [70]:
chose_team("Auburn","Jacksonville State")

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

Auburn  won!


### Round 2

In [71]:
chose_team("Gonzaga","Memphis")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Gonzaga  won!


In [73]:
chose_team("New Mexico State","Arkansas")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Arkansas  won!


In [74]:
chose_team("Alabama","Montana State")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Montana State  won!


In [75]:
chose_team("Michigan State","Cal State Fullerton")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Cal State Fullerton  won!


In [76]:
chose_team("Norfolk State","Marquette")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


Norfolk State  won!


'Norfolk State'

In [77]:
chose_team("Saint Mary's","UCLA")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
UCLA  won!


In [78]:
chose_team("Texas","Purdue")

Purdue  won!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


'Purdue'

In [79]:
chose_team("Murray State","Kentucky")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Kentucky  won!


In [80]:
chose_team("Arizona","Seton Hall")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Seton Hall  won!


In [81]:
chose_team("Houston","Illinois")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


Houston  won!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


'Houston'

In [82]:
chose_team("Colorado State","Tennessee")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they we

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Colorado State  won!


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


In [83]:
chose_team("Ohio State","Delaware")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Ohio State  won!


In [84]:
chose_team("Kansas","Providence")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Providence  won!


In [85]:
chose_team("Wisconsin","USC")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
USC  won!


### Round 4

In [86]:
chose_team("Gonzaga","Arkansas")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Gonzaga  won!


In [88]:
chose_team("Montana State","Cal State Fullerton")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Cal State Fullerton  won!


In [89]:
chose_team("Norfolk State","UCLA")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Norfolk State  won!


In [90]:
chose_team("Purdue","Kentucky")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/di

This is a double tossup
Kentucky  won!


In [91]:
chose_team("Seton Hall","Houston")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Seton Hall  won!


In [92]:
chose_team("Colorado State","Ohio State")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they we

This is a tossup
This is a double tossup
Colorado State  won!


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


In [93]:
chose_team("Providence","USC")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
USC  won!


### Round 5

In [94]:
chose_team("Seton Hall","Colorado State")

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they we

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Colorado State  won!


In [95]:
chose_team("Norfolk State","Kentucky")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Norfolk State  won!


In [96]:
chose_team("Gonzaga","Cal State Fullerton")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Gonzaga  won!


In [97]:
chose_team("Gonzaga","Norfolk State")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Gonzaga  won!


In [98]:
chose_team("Colorado State","USC")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they we

This is a tossup
This is a double tossup
Colorado State  won!


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


In [99]:
chose_team("Gonzaga","Colorado State")

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Opp BenchOther 
- Opp Def FTR%Other 
- Opp Def ORB%Other 
- Opp Def TOP%Other 
- Opp Def eFG%Other 
- ...
Feature names must be in the same order as they we

This is a tossup


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


This is a double tossup
Colorado State  won!
